In [1]:
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
pip install transformers huggingface_hub

In [3]:
from google.colab import userdata
# userdata.get("HF_TOKEN")

In [4]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HF_TOKEN")

In [5]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [ ]:
from langchain import HuggingFaceHub
repo_id="mistralai/Mistral-7B-Instruct-v0.3"

In [ ]:
llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64})

<ipython-input-28-105c34be3637>:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64})


In [ ]:
llm.invoke("What is the capital of France?")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


'What is the capital of France? Paris\n\nThe capital of France is Paris. It is the largest city in the country and is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris has been the capital of France since 508 AD and is a major global center for art, fashion, gastronomy, and culture. It is also the most populated city in France, with over 2 million residents within its administrative limits and over 12 million people in its metropolitan area.'

In [ ]:
from langchain_huggingface import HuggingFacePipeline

In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [ ]:
# model_id = "gpt2"
model_id="mistralai/Mistral-7B-Instruct-v0.3"

In [ ]:
"""
main_chat_wtih_native_code.py

This module loads a local causal language model, generates responses based on user input,
and logs the thinking process along with the final answer.

The model is loaded with quantization settings to optimize memory usage and performance
(32 bits -> 8 bits).

The orchestration of the process is handled by the main function, and individual steps are
broken down into dedicated functions.
"""

import logging
import os
import time
from dotenv import load_dotenv
from typing import Tuple
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig


def load_model() -> Tuple[AutoTokenizer, AutoModelForCausalLM]:
    """
    Load the model and tokenizer with 8-bit quantization configuration to optimize memory usage
    and inference performance.

    This function loads the model and tokenizer dynamically based on the environment variables.
    It also configures the model for 8-bit quantization, which reduces the memory footprint
    and speeds up inference, with the additional flexibility to control how much of the model
    is quantized via the 'llm_int8_threshold'.

    The quantization is performed using the `BitsAndBytesConfig`:
    - `load_in_8bit=True` ensures that the model weights are loaded in 8-bit precision (INT8),
        reducing the model's memory requirements.
    - `llm_int8_threshold=6.0` specifies a threshold for applying 8-bit quantization.
    Weights with magnitudes larger than this threshold will be quantized to 8-bit precision,
    while smaller weights may remain in higher precision to retain accuracy.

    The model is loaded in a way that allows it to automatically balance between CPU
    and GPU resources.

    Returns:
        Tuple: The tokenizer and model objects.
    """
    load_dotenv()
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
    print(f"Loading model: {model_name}")

    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Define the quantization configuration for 8-bit
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0
        )

    # Load model
    config = AutoConfig.from_pretrained(
        model_name,
        device_map="auto",  #! Dynamically balancing between CPU and GPU
        trust_remote_code=True,
    )

    if hasattr(config, "quantization_config"):
        delattr(config, "quantization_config")

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        config=config,
        trust_remote_code=True,
    )

    print(f"Model ({model_name}) loaded.")
    return tokenizer, model


def generate_chat_response(
    prompt: str,
    tokenizer: AutoTokenizer,
    model: AutoModelForCausalLM,
    max_length: int = 2000,
    temperature: float = 0.7,
    top_k: int = 50,
    top_p: float = 0.9,
) -> Tuple[str, str]:
    """
        Generate a response from the model based on the input prompt.

        Args:
            - prompt (str): The input prompt.
            - tokenizer (AutoTokenizer): The tokenizer to preprocess the input.
            - model (AutoModelForCausalLM): The model used for generating the response.
            - max_length (int): The maximum length of the generated output.
            - temperature (float): The randomness of the output.
            - top_k (int): The number of top token choices.
            - top_p (float): The cumulative probability threshold for nucleus sampling.

        Returns:
            Tuple[str, str]: The thinking steps and the final answer from the model.

    .   #* About temp, top_k, top_p
        Temperature controls the randomness of the generated text, with higher values
        leading to more creative but less coherent output, and lower values resulting
        in more predictable, deterministic responses.

        Top-k limits token choices to the top k most likely options, reducing irrelevant
        text but potentially limiting creativity.

        Top-p (nucleus sampling) selects tokens dynamically until a cumulative probability
        threshold is met, balancing diversity and coherence, often used in combination
        with top-k.
    """

    # Add a "thinking" instruction to the prompt
    thinking_prompt = f"""
    Question: {prompt}
    <think>
    Please reason through the problem step by step without repeating yourself. \
Each step should be concise and progress logically toward the final answer:
    """

    # Tokenize the input prompt
    inputs = tokenizer(thinking_prompt, return_tensors="pt")

    # Move input tensors to the same device as the model
    device = next(model.parameters()).device
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Start timing the thinking process
    start_time = time.time()

    # Generate logits and outputs
    with torch.no_grad():
        logits = model(**inputs).logits
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,  # have multi-options (tokens) picks 1 based on prob.
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
        )

        print(
            f"Intermediate logits shape: {logits.shape}"
        )  # Debugging: inspect logits

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    time_str = f"{int(minutes):02}:{int(seconds):02}"

    # Decode the full response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the "thinking" part and the final answer
    if "<think>" in full_response and "</think>" in full_response:
        thinking_part = full_response.split("<think>")[1].split("</think>")[0].strip()
        final_answer = full_response.split("</think>")[1].strip()
    else:
        thinking_part = "No thinking steps captured."
        final_answer = full_response

    # Log the thinking steps and final answer
    print(f"Thinking time: {time_str}")
    print(f"\nThinking Steps:\n{thinking_part}")
    print(f"\nFinal Answer:\n{final_answer}")

    return thinking_part, final_answer


def main():
    """Orchestrate the loop"""
    print("Chat with DeepSeek R1! Type 'exit' to end the chat.")

    # Load the model and tokenizer
    tokenizer, model = load_model()

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            break

        # Generate and display the response
        thinking_output, final_output = generate_chat_response(
            prompt=user_input, tokenizer=tokenizer, model=model, max_length=2000
        )
        print(f"DeepSeek (Thinking Steps): {thinking_output}")
        print(f"DeepSeek (Final Answer): {final_output}")
        print(f"Response: {thinking_output} | {final_output}")


if __name__ == "__main__":
    main()

Chat with DeepSeek R1! Type 'exit' to end the chat.
Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model (deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B) loaded.

You: Who is dalai lama ?


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Intermediate logits shape: torch.Size([1, 42, 151936])
Thinking time: 01:54

Thinking Steps:
Please reason through the problem step by step without repeating yourself. Each step should be concise and progress logically toward the final answer:
     - The user is asking about Dalai Lama.
     - Dalai Lama is a political figure who has been involved in various conflicts and protests in Tibet.
     - He is known for his involvement in the Traction Movement and the widely criticized anti-independence movement.
     - His actions have led to a lot of tension in the region.
     - It's important to note his role in the political landscape and the impact his actions have had.
     - Additionally, there is concern about his potential for change or further actions in the region.
     - The answer should be clear and concise, addressing both his role and the broader implications.

Final Answer:
Dalai Lama is a political figure who has been involved in various conflicts and protests in Tibet. He 

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who is illuminati?"

print(llm_chain.run(question))

<ipython-input-29-890ab8721e5e>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)
<ipython-input-29-890ab8721e5e>:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_chain.run(question))
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Question: Who is illuminati?

Answer: Let's think step by step. First, we need to understand what illuminati means. Illuminati is a term used to describe a group of people who are believed to possess secret knowledge or powers. They are often associated with conspiracy theories and secret societies.

Now, let's break down the name "illuminati" into its components:

1. "Illu" - This comes from the Latin word "illuminatus," which means "enlightened" or "illuminated."
2. "Minati" - This is a suffix that often indicates a group or organization.

So, when we put these together, "illuminati" literally means "the enlightened ones" or "the illuminated ones."

However, the term has taken on a more specific meaning in popular culture. It's often used to refer to a secret society that is believed to control world affairs. This idea is most famously depicted in the video game series "Assassin's Creed," where the Assassins and the Templars are engaged in a centuries-old conflict with the Illuminati

In [6]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain import HuggingFaceHub
import torch


model_id = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
llm = HuggingFacePipeline(pipeline=pipeline)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
llm.invoke("hello how are you ?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"hello how are you ?\n\nYou don't miss much, and you never know."

In [8]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Let's think step by step and give a deep answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who is Rajnikant?"

print(llm_chain.run(question))

<ipython-input-8-6a2a494d11f2>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)
<ipython-input-8-6a2a494d11f2>:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_chain.run(question))
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: Who is Rajnikant?

Let's think step by step and give a deep answer. First, let's think about who is who.

Let's start with Rajnikant.

Rajnikant was on the list of people that he's not really famous for and who wasn't particularly fond of him to begin with. He also didn't have a lot of reputation as a guy with the balls of a million.

At this point you need to keep in mind that Rajnikant would be a true fan of many aspects of the game.

However
